---
디시전 트리로 추천/비추천 예측
방법 1. User마다 decision tree 모델을 하나씩 갖는다.
방법 2. user각자 고려하지 않고, user_id도 feature로 사용한다

X - games features
Y - 추천 여부

In [1]:

import numpy as np
import pandas as pd

# Load the data
data = pd.read_csv('../datasets/merged_steam_games.csv')

# convert date_realese columns to unixtime
data['release_date'] = pd.to_datetime(data['release_date'])
data['release_date'] = data['release_date'].apply(lambda x: int(x.timestamp()))

data['date'] = pd.to_datetime(data['date'])
data['date'] = data['date'].apply(lambda x: int(x.timestamp()))

ids = pd.DataFrame(data['app_id'])

# 전처리 안된 column 제거
data = data.drop(['categories'], axis=1)

print(data.shape)
print(data.head())
print(data.columns)
print(data.dtypes)
print(data.describe())


(9764026, 54)
   app_id  helpful  funny        date  is_recommended  hours   user_id  win  \
0  304390        4      0  1619308800               0   42.5  10178125    1   
1  304390        0      0  1612310400               1  265.8   9545809    1   
2  304390        0      0  1604880000               0  493.3   2968763    1   
3  304390        0      0  1624579200               1  114.5  13598296    1   
4  304390        0      0  1617408000               0  226.0   6549932    1   

   mac  linux  ...  RPG  Indie  Software Training  Simulation  \
0    0      0  ...    0      0                  0           0   
1    0      0  ...    0      0                  0           0   
2    0      0  ...    0      0                  0           0   
3    0      0  ...    0      0                  0           0   
4    0      0  ...    0      0                  0           0   

   Game Development  Massively Multiplayer  Early Access  Nudity  Strategy  \
0                 0                      0

In [2]:
# 데이터셋에서 게임 데이터 분할
recommend_columns = ['app_id','helpful', 'funny', 'date', 'is_recommended', 'hours',
       'user_id']

recommend_datas = data[recommend_columns]

game_infos = data.drop(recommend_columns[1:], axis=1)

## Scailing

In [3]:
from sklearn.preprocessing import StandardScaler
scaled_data = game_infos.copy()
scale_cols = ['release_date','positive_ratio','user_reviews','price_final','price_original', 'required_age', 'achievements', 'positive_ratings', 'negative_ratings', 
              'average_playtime', 'median_playtime', 'owners', 'price',]
# scale_cols = [col for col in scaled_data.columns if col not in no_scale_cols]
unique_values = ids['app_id'].unique()
print(unique_values)

index_positions = [ids[ids['app_id'] == value].index[0] for value in unique_values]

scaled_data = game_infos.iloc[index_positions]

# Scale the data
scaler = StandardScaler()
scaled_data[scale_cols] = scaler.fit_transform(scaled_data[scale_cols])

[304390 306130 238960 ... 431700 563200 813650]


/var/folders/tr/1pblcvrd48j5lfn0khj_hm4w0000gn/T/ipykernel_37371/2006423257.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_data[scale_cols] = scaler.fit_transform(scaled_data[scale_cols])


In [4]:
# 스케일 된 게임 데이터와 추천 데이터 다시 합치기
merged_data = pd.merge(recommend_datas, scaled_data, on='app_id')
print(merged_data.shape)

(9764026, 54)


## 테스트 데이터 분리

In [6]:
from sklearn.model_selection import train_test_split

Y = merged_data['is_recommended']
X = merged_data.drop(['is_recommended'], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=156)

X_colums = merged_data.columns.drop(['is_recommended'])
df_X_train = pd.DataFrame(X_train , columns=X_colums)
df_Y_train = pd.DataFrame(Y_train, columns=['is_recommended'])
df_X_test = pd.DataFrame(X_test, columns=X_colums)
df_Y_test = pd.DataFrame(Y_test, columns=['is_recommended'])

In [34]:

from sklearn.tree import DecisionTreeClassifier

# 특정 유저 아이디에 대한 추천 게임 예측
# 각 유저별로 decision tree를 만들어서 예측

# 유저 아이디를 입력받아서 해당 유저의 추천 데이터만 추출
target_user_id = 10178125

train_indices = df_X_train.index[df_X_train['user_id'] == target_user_id].tolist()
X_target_train = df_X_train.loc[train_indices]
Y_target_train = df_Y_train.loc[train_indices]

test_indices = df_X_test.index[df_X_test['user_id'] == target_user_id].tolist()
X_target_test = df_X_test.loc[test_indices]
Y_target_test = df_Y_test.loc[test_indices]

# 모델 학습
model = DecisionTreeClassifier(max_depth=5, random_state=0)
model.fit(X_target_train, Y_target_train)

#예측
print(model.predict(X_target_test))
print(model.score(X_target_train, Y_target_train))

[0 0 1]
1.0
